In [2]:
import pandas

In [3]:
import pandas as pd

data = pd.read_parquet("../data/transformed/ts_data_rides_2023_02.parquet")
data.head()

,pickup_hour,rides,pickup_location_id
0,2023-02-01 00:00:00,4,4
1,2023-02-01 01:00:00,7,4
2,2023-02-01 02:00:00,0,4
3,2023-02-01 03:00:00,0,4
4,2023-02-01 04:00:00,0,4


In [4]:
ts_data_one_location = data.loc[data['pickup_location_id']==43,:].reset_index(drop=True)
ts_data_one_location.head()

,pickup_hour,rides,pickup_location_id
0,2023-02-01 00:00:00,7,43
1,2023-02-01 01:00:00,1,43
2,2023-02-01 02:00:00,1,43
3,2023-02-01 03:00:00,1,43
4,2023-02-01 04:00:00,3,43


In [5]:
def get_cutoff_indices(
    data:pd.DataFrame,
    n_features:int,
    step_size:int
) ->list:

    stop_position = len(data) - 1

    # start the first sub-sequence at index postion 0
    subseq_first_idx= 0
    subseq_mid_idx = n_features
    subseq_last_idx = n_features +1

    indices = []

    while subseq_last_idx <= stop_position:
        indices.append((subseq_first_idx, subseq_mid_idx, subseq_last_idx))

        subseq_first_idx += step_size
        subseq_mid_idx += step_size
        subseq_last_idx += step_size
    
    return indices





In [6]:
n_features =24
step_size = 1

indices = get_cutoff_indices(ts_data_one_location,n_features=n_features,step_size=step_size)

In [7]:
indices[:5]

[(0, 24, 25), (1, 25, 26), (2, 26, 27), (3, 27, 28), (4, 28, 29)]

In [8]:
import numpy as np

n_examples = len(indices)
x = np.ndarray(shape= (n_examples, n_features), dtype= np.float32)
y = np.ndarray(shape =(n_examples), dtype=np.float32)
pickup_hours = []

for i,idx in enumerate(indices):
    x[i,:] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
    y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
    pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

In [9]:
print(f'{x.shape= }')
print(f'{x= }')
print(f'{pickup_hours[:5]}')

x.shape= (647, 24)
x= array([[  7.,   1.,   1., ...,  57.,  40.,  25.],
       [  1.,   1.,   1., ...,  40.,  25.,  10.],
       [  1.,   1.,   3., ...,  25.,  10.,   1.],
       ...,
       [ 59.,  57.,  27., ..., 157.,  80.,  70.],
       [ 57.,  27.,   4., ...,  80.,  70.,  76.],
       [ 27.,   4.,   2., ...,  70.,  76.,  90.]], dtype=float32)
[Timestamp('2023-02-02 00:00:00'), Timestamp('2023-02-02 01:00:00'), Timestamp('2023-02-02 02:00:00'), Timestamp('2023-02-02 03:00:00'), Timestamp('2023-02-02 04:00:00')]


In [10]:
features_one_location = pd.DataFrame(
    x,
    columns= [f'rides_previous_{i+1}_hour' for i in reversed(range(n_features))]
)
features_one_location

,rides_previous_24_hour,rides_previous_23_hour,rides_previous_22_hour,rides_previous_21_hour,rides_previous_20_hour,rides_previous_19_hour,rides_previous_18_hour,rides_previous_17_hour,rides_previous_16_hour,rides_previous_15_hour,...,rides_previous_10_hour,rides_previous_9_hour,rides_previous_8_hour,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour
0,7.0,1.0,1.0,1.0,3.0,3.0,18.0,56.0,65.0,74.0,...,108.0,100.0,138.0,112.0,104.0,103.0,78.0,57.0,40.0,25.0
1,1.0,1.0,1.0,3.0,3.0,18.0,56.0,65.0,74.0,116.0,...,100.0,138.0,112.0,104.0,103.0,78.0,57.0,40.0,25.0,10.0
2,1.0,1.0,3.0,3.0,18.0,56.0,65.0,74.0,116.0,113.0,...,138.0,112.0,104.0,103.0,78.0,57.0,40.0,25.0,10.0,1.0
3,1.0,3.0,3.0,18.0,56.0,65.0,74.0,116.0,113.0,139.0,...,112.0,104.0,103.0,78.0,57.0,40.0,25.0,10.0,1.0,1.0
4,3.0,3.0,18.0,56.0,65.0,74.0,116.0,113.0,139.0,115.0,...,104.0,103.0,78.0,57.0,40.0,25.0,10.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,95.0,45.0,59.0,57.0,27.0,4.0,2.0,2.0,0.0,2.0,...,61.0,68.0,87.0,91.0,121.0,108.0,135.0,151.0,146.0,157.0
643,45.0,59.0,57.0,27.0,4.0,2.0,2.0,0.0,2.0,0.0,...,68.0,87.0,91.0,121.0,108.0,135.0,151.0,146.0,157.0,80.0
644,59.0,57.0,27.0,4.0,2.0,2.0,0.0,2.0,0.0,2.0,...,87.0,91.0,121.0,108.0,135.0,151.0,146.0,157.0,80.0,70.0
645,57.0,27.0,4.0,2.0,2.0,0.0,2.0,0.0,2.0,10.0,...,91.0,121.0,108.0,135.0,151.0,146.0,157.0,80.0,70.0,76.0


In [11]:
targets_one_location= pd.DataFrame(y, columns=[f'target_rides_next_hour'])
targets_one_location

,target_rides_next_hour
0,10.0
1,1.0
2,1.0
3,0.0
4,1.0
...,...
642,80.0
643,70.0
644,76.0
645,90.0


In [12]:
from tqdm import tqdm
import pandas as pd
import numpy as np

def transform_ts_data_into_features_and_target(
    ts_data: pd.DataFrame, 
    input_seq_len: int, 
    step_size: int) -> pd.DataFrame:
    """
    Slices and transposes data from time-series format to a (features, target)
    format that we can use to train Supervised ML models.
    """

    assert set(ts_data.columns) == {'pickup_hour', 'rides', 'pickup_location_id'}
    location_ids = ts_data['pickup_location_id'].unique()
    features = pd.DataFrame()
    targets = pd.DataFrame()
    for location_id in tqdm(location_ids):
        ts_data_one_location = ts_data.loc[ts_data.pickup_location_id == location_id,
                                           ['pickup_hour', 'rides']]

        indices = get_cutoff_indices(
            ts_data_one_location,
            input_seq_len,
            step_size
        )

        n_examples = len(indices)
        x = np.ndarray(shape=(n_examples, input_seq_len), dtype=np.float32)
        y = np.ndarray(shape=(n_examples), dtype=np.float32)
        pickup_hours = []
        for i, idx in enumerate(indices):
            x[i, :] = ts_data_one_location.iloc[idx[0]:idx[1]]['rides'].values
            y[i] = ts_data_one_location.iloc[idx[1]:idx[2]]['rides'].values
            pickup_hours.append(ts_data_one_location.iloc[idx[1]]['pickup_hour'])

        # numpy -> pandas
        features_one_location = pd.DataFrame(
            x,
            columns=[f'rides_previous_{i+1}_hour' for i in reversed(range(input_seq_len))]
        )

        features_one_location['pickup_hour'] = pickup_hours
        features_one_location['pickup_location_id'] = location_id

        # numpy -> pandas
        targets_one_location = pd.DataFrame(y, columns=['target_rides_next_hour'])

        # concatenate results
        features = pd.concat([features, features_one_location])
        targets = pd.concat([targets, targets_one_location])

    features.reset_index(inplace=True, drop=True)
    targets.reset_index(inplace=True, drop=True)

    return features, targets['target_rides_next_hour']

In [13]:
features, targets = transform_ts_data_into_features_and_target(
    data,
    input_seq_len=24*7*1 , #one week of history
    step_size=24

)

print(f'{features.shape=}')

print(f'{targets.shape=}')

100%|██████████| 258/258 [00:00<00:00, 292.52it/s]

features.shape=(5418, 170)
targets.shape=(5418,)
